<a href="https://colab.research.google.com/github/BonnieAo/SignatureWork/blob/main/SW_SampleData_Virtualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 1: Data Wrangling

SQL code for data querying on Google BigQuery:

Contract data:

"SELECT address,is_erc20,is_erc721,block_timestamp,block_number,block_hash 

FROM `bigquery-public-data.crypto_ethereum.contracts` 

ORDER BY block_timestamp DESC"

Transaction data:

"SELECT receipt_contract_address, nonce,from_address, to_address, value, gas, gas_price, receipt_cumulative_gas_used,receipt_gas_used,receipt_status,block_timestamp 

FROM `bigquery-public-data.crypto_ethereum.transactions` 

WHERE receipt_contract_address IS NOT NULL

ORDER BY block_timestamp DESC"



In [ ]:
import pandas as pd 
import numpy as np
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn import metrics 
import matplotlib.pyplot as plt

In [ ]:
###show all columns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
transaction=pd.read_csv('https://raw.githubusercontent.com/BonnieAo/SignatureWork/main/transaction_contractcreated_16000.csv')
transaction["contract_address"] = transaction['receipt_contract_address']
transaction.head()

In [ ]:
contract=pd.read_csv('https://raw.githubusercontent.com/BonnieAo/SignatureWork/main/contract_latest_16000.csv')
contract['contract_address'] = contract["address"]
contract.head()

# Transaction Data Virtualization

In [ ]:
#Show the proportion of receipt status (success/failure)
import matplotlib.pyplot as plt
labels = ['success','failure']
success = (len(transaction[transaction['receipt_status']==1])/len(transaction))*100
failure = 100-success
x = [success,failure]
explode = (0,0.1) 
pie = plt.pie(x,explode= explode,labels=labels,autopct='%3.2f%%')
plt.axis('equal')
plt.show()

In [ ]:
#Show the distribution of gas provided by the sender
plt.hist(transaction["gas"],10,histtype='stepfilled',facecolor='b',alpha=0.75) 
plt.title('Histogram of gas provided by the sender') 
plt.show()

In [ ]:
#Show the distribution of gas price provided by the sender in Wei
plt.hist(transaction["gas_price"],10,histtype='stepfilled',facecolor='b',alpha=0.75) 
plt.title('Histogram of gas price provided by the sender') 
plt.show()

In [ ]:
#Show the distribution of gas used when the trsnsaction is excuted in the block
plt.hist(transaction["receipt_cumulative_gas_used"],10,histtype='stepfilled',facecolor='b',alpha=0.75) 
plt.title('Histogram of gas used when the trsnsaction is excuted in the block') 
plt.show()

In [ ]:
#Show the distribution of gas used by this specific transaction alone
plt.hist(transaction["receipt_gas_used"],10,histtype='stepfilled',facecolor='b',alpha=0.75) 
plt.title('Histogram of gas used by this specific transaction alone') 
plt.show()

# Contract Data Virtualization

In [ ]:
#Count the number of contract address in this sample data
grouped_contract_address = contract.groupby(contract['contract_address'])
len(grouped_contract_address)

# Contract & Transaction Data Merge

In [ ]:
df = pd.merge(transaction,contract,how='inner',on="contract_address")
df.head()

In [ ]:
#Count the number of rows in this merged data
len(df)

In [ ]:
#Show the scatter plot of receipt_cumulative_gas_used vs. receipt_gas_used
plt.scatter(df['receipt_cumulative_gas_used'],df['receipt_gas_used'])
plt.title("receipt_cumulative_gas_used vs. receipt_gas_used")
plt.show()